Thanks to all the contributors with kernels to this competition!

Specially, to the ones that provided code and comments that inspired us making this and other kernels we developed in the context of this competition:

https://www.kaggle.com/dmdm02/complete-eda-voting-lightgbm

https://www.kaggle.com/chocozzz/two-sigma-news-simple-eda-prophet-nlp

https://www.kaggle.com/ashishpatel26/bird-eye-view-of-two-sigma-nn-approach

https://www.kaggle.com/jsaguiar/baseline-with-news

https://www.kaggle.com/artgor/eda-feature-engineering-and-everything

https://www.kaggle.com/christofhenkel/market-data-nn-baseline

https://www.kaggle.com/smasar/tutorial-timeseriesapproach

https://www.kaggle.com/rabaman/0-64-in-100-lines

https://www.kaggle.com/guowenrui/market-nn-if-you-like-you-can-use-it-and-upvote/notebook

https://www.kaggle.com/smasar/eda-preprocessing-processing-evaluation

Importar librerías

In [ ]:
import pandas as pd
import numpy as np
import time
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

plt.style.use('seaborn')
sns.set(font_scale=2)

import random
def generate_color():
    color = '#{:02x}{:02x}{:02x}'.format(*map(lambda x: random.randint(0, 255), range(3)))
    return color

import warnings 
warnings.filterwarnings('ignore')
import os

Crear entorno

In [ ]:
from kaggle.competitions import twosigmanews
env = twosigmanews.make_env()
print('Done!')

Carga de los datos de training

In [ ]:
(market_train_df, news_train_df) = env.get_training_data()

Dadas las características del proyecto, de cara a poder medir la accuracy, simplemente consultamos las características de los ficheros de test... pero no vamos a usarlos en lo sucesivo, sino que vamos a coger un subconjunto de los ficheros de train, que tengan las mismas características: los datos de mercados del último día y los datos del fichero de noticias posteriores a las 22h del penúltimo día (más todos los del último día). 

In [ ]:
#del market_test_df, news_test_df

split = '2016-12-29 22:00:00+00:00'

market_test_df = market_train_df.loc[market_train_df['time'] > split]
news_test_df = news_train_df.loc[news_train_df['time'] > split]

market_train_df = market_train_df.loc[market_train_df['time'] <= split].reset_index(drop=True)
news_train_df = news_train_df.loc[news_train_df['time'] <= split].reset_index(drop=True)

In [ ]:
print("In market_train_df: ", market_train_df.shape);print("In market_test_df: ", market_test_df.shape);
print("In news_train_df: ", news_train_df.shape);print("In news_test_df: ", news_test_df.shape)

## 5.1.3 Análisis relacional entre variables

Antes de nada, vamos a ver las matrices de correlación entre las variables de cada uno de los ficheros, como punto de partida para plantearnos evaluar relaciones cruzadas entre variables.

Primero, la del fichero de mercados:

In [ ]:
# Plot correlation
columns_corr = ['returnsOpenNextMktres10', 'volume', 'close', 'open', 'returnsClosePrevRaw1', 'returnsOpenPrevRaw1', 'returnsClosePrevRaw10', 'returnsOpenPrevRaw10',
           'returnsClosePrevMktres1', 'returnsOpenPrevMktres1', 'returnsClosePrevMktres10', 'returnsOpenPrevMktres10']
colormap = plt.cm.RdBu
plt.figure(figsize=(18,15))
sns.heatmap(market_train_df[columns_corr].astype(float).corr(), linewidths=0.1, vmax=1.0, vmin=-1., square=True, cmap=colormap, linecolor='white', annot=True)
plt.title('market_train_df correlation matrix')

Sólo mirando la matriz de correlación del fichero de mercados, se pueden extraer varias conclusiones:

- hay una altísima correlación entre las variables "open" y "close". Lo cuál parece bastante lógico, ya que no debería ser muy frecuente variaciones muy fuertes en la cotización dentro de una misma sesión.
- hay también muy altas correlaciones entre las variables "Raw" y las variables "Mktres" correspondientes. Esto también parece bastante lógico, pues debería estar bastante relacionado: el rendimiento interdiario al cierre en crudo, con el rendimiento interdiario al cierre ajustado al mercado, el rendimiento inter-diez-días al cierre en crudo, con el rendimiento inter-diez-días al cierre ajustado al mercado, ... y así para todas las "Raw"/"Mktres".
- hay algunas relaciones también significativas, aunque no tan altas, entre:
    a. returnClosePrevRaw1 y returnClosePrevRaw10
    b. returnClosePrevRaw1 y returnClosePrevMktres10
    c. returnClosePrevMktres1 y returnClosePrevRaw10
    d. returnClosePrevMktres1 y returnClosePrevMktres10
  lo cuál también tiene una cierta lógica también, ya que lo "normal" sería que los rendimientos a un día y a 10 días estén relacionados.
- esas mismas relaciones significativas, no se presentan entre las variables "Open".
- y, sobre todo, llama la atención que no hay ninguna relación significativa entre la variable objetivo y ninguna otra variable.


La conclusión sobre la baja correlación de la variable objetivo con el resto de variables nos preocupa, así que decidimos hacer una prueba para ver si podemos detectar alguna relación significativa que no se detecte a primera vista.

Teniendo en cuenta de lo que estamos hablando, mercados... y del horizonte temporal que estamos manejando, desde 2007... pensamos que el periodo de la crisis financiera puede estar causando cierta distorsión.

Veamos la evolución temporal de las cotizaciones al cierre, por cuantiles:

In [ ]:
data = []
#market_train_df['close'] = market_train_df['close'] / 20
for i in [0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95]:
    price_df = market_train_df.groupby('time')['close'].quantile(i).reset_index()

    data.append(go.Scatter(
        x = price_df['time'].dt.strftime(date_format='%Y-%m-%d').values,
        y = price_df['close'].values,
        name = f'{i} quantile'
    ))
layout = go.Layout(dict(title = "Trends of closing prices by quantiles",
                  xaxis = dict(title = 'Month'),
                  yaxis = dict(title = 'Price (USD)'),
                  ),legend=dict(
                orientation="h"))
py.iplot(dict(data=data, layout=layout), filename='basic-line')

Lo primero que llama la atención, efectivamente, es la caída de las cotizaciones entre 2008 y 2009.

Se pueden apreciar más bajadas significativas (principipalmente se ven 4)... pero lo que se aprecia entre 2008 y 2009, como ya sabíamos previamente, es realmente algo sin parangón.


Otro dato curioso es que se aprecia cómo lo cuantiles altos tienen una clara tendencia a la alza (desde 2010), mientras que cuantiles bajos no.

Podría deberse a la entrada de más compañías pequeñas en el mercado, cuyo precio por acción es más bajo que el de las grandes compañías más asentadas.


Ahondemos más en el efecto de la crisis, echándole un vistazo a la variable objetivo

In [ ]:
data = []
for i in [0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95]:
    price_df = market_train_df.groupby('time')['returnsOpenNextMktres10'].quantile(i).reset_index()

    data.append(go.Scatter(
        x = price_df['time'].dt.strftime(date_format='%Y-%m-%d').values,
        y = price_df['returnsOpenNextMktres10'].values,
        name = f'{i} quantile'
    ))
layout = go.Layout(dict(title = "Trends of returnsOpenNextMktres10 by quantiles",
                  xaxis = dict(title = 'Month'),
                  yaxis = dict(title = 'Returns (USD)'),
                  ),legend=dict(
                orientation="h"),)
py.iplot(dict(data=data, layout=layout), filename='basic-line')

Podemos ver como, a pesar de haber muchas fluctuaciones... lo que era de esperar, las fluctuaciones en 2009 son algo fuera de lo común.

Eliminemos todo lo anterior a 2010 y volvamos a ver la evolución de la variable objetivo:

In [ ]:
data = []
market_train_df = market_train_df.loc[market_train_df['time'] >= '2010-01-01 22:00:00+0000']

price_df = market_train_df.groupby('time')['returnsOpenNextMktres10'].mean().reset_index()

data.append(go.Scatter(
    x = price_df['time'].dt.strftime(date_format='%Y-%m-%d').values,
    y = price_df['returnsOpenNextMktres10'].values,
    name = f'{i} quantile'
))
layout = go.Layout(dict(title = "Trends of returnsOpenNextMktres10 mean",
                  xaxis = dict(title = 'Month'),
                  yaxis = dict(title = 'Returns (USD)'),
                  ),legend=dict(
                orientation="h"),)
py.iplot(dict(data=data, layout=layout), filename='basic-line')

Aunque sigue habiendo muchas fluctuaciones, como obviamente ocurre en los mercados, se trata de un conjunto de datos con un comportamiento mucho más estable.

Esto nos hace plantearnos la siguiente pregunta:
- deberíamos conservar todos los datos de los que disponemos, para el posterior análisis?
- o deberíamos quedarnos con el conjunto de datos más estable... y predecible?

No hay una respuesta buena, pero para decidir lo condicionamos a lo siguiente...
creemos que vamos a predecir un periodo de las características de la crisis financiera?

La verdad es que no... con lo que decidimos que, en adelante, vamos a quedarnos únicamente con los datos desde 2010.


Ya tenemos filtrado el fichero de mercados, así que... antes de que se nos olvide, vamos a filtrar también el fichero de noticias... aunque, en este caso, tomaremos los datos desde el final de la sesión anterior... esto es, desde las 22h del último día de 2009.

In [ ]:
news_train_df = news_train_df.loc[news_train_df['time'] >= '2009-12-31 22:00:00+0000']

Volviendo al fichero de mercados, veamos nuevamente la matriz de correlación... una vez eliminados los datos de la crisis de 2007-2009.

In [ ]:
# Plot correlation
columns_corr = ['returnsOpenNextMktres10', 'volume', 'close', 'open', 'returnsClosePrevRaw1', 'returnsOpenPrevRaw1', 'returnsClosePrevRaw10', 'returnsOpenPrevRaw10',
           'returnsClosePrevMktres1', 'returnsOpenPrevMktres1', 'returnsClosePrevMktres10', 'returnsOpenPrevMktres10']
colormap = plt.cm.RdBu
plt.figure(figsize=(18,15))
sns.heatmap(market_train_df[columns_corr].astype(float).corr(), linewidths=0.1, vmax=1.0, vmin=-1., square=True, cmap=colormap, linecolor='white', annot=True)
plt.title('market_train_df correlation matrix')

Ahora, la matriz de correlación del fichero de mercados muestra mayores correlaciones:

- sigue viéndose una altísima correlación entre las variables "open" y "close".
- siguen viéndose muy altas correlaciones entre las variables "Raw" y las variables "Mktres" correspondientes. De hecho, incluso aumentan.
- se mantienen relaciones también significativas, aunque no tan altas, entre:
    a. returnClosePrevRaw1 y returnClosePrevRaw10
    b. returnClosePrevRaw1 y returnClosePrevMktres10
    c. returnClosePrevMktres1 y returnClosePrevRaw10
    d. returnClosePrevMktres1 y returnClosePrevMktres10
- pero esas correlaciones son aún más significativas entre las variables "Open":
    a. returnOpenPrevRaw1 y returnOpenPrevRaw10
    b. returnOpenPrevRaw1 y returnOpenPrevMktres10
    c. returnOpenPrevMktres1 y returnOpenPrevRaw10
    d. returnOpenPrevMktres1 y returnOpenPrevMktres10
- e incluso aparece alguna correlación significativa entre variable "Close" y "Open".
- y, sobre todo, llama la atención que aparecen algunas correlaciones superiores al 10% entre la variable objetivo y las variables "Open": returnOpenPrevRaw1, returnOpenPrevRaw10, returnOpenPrevMktres1 y returnOpenPrevMktres10.

Esto nos reafirma en que el subconjunto de datos seleccionado es más apropiado y nos va a permitir realizar mejores predicciones.


Ahora, a modo ilustrativo, vamos a seleccionar un único activo y a ver la matriz de correlaciones para ese único activo.

Para ello, vamos a comprobar la lista de activos con mayor volúmen de cotización y seleccionar de este modo uno que resulte significativo.

In [ ]:
assetCode_df = market_train_df.groupby('assetCode')['volume'].sum().sort_values(ascending=False)
assetCode_df[:15].plot.barh()
plt.ylabel('assetCode')
plt.xlabel('Trading volume')
plt.title('Top 15 asset code by volume')
plt.gca().invert_yaxis()

Para que resulte más inteligible, probaremos por el nombre:

In [ ]:
assetName_Volume = market_train_df.groupby('assetName')['volume'].sum().sort_values(ascending=False)
assetName_Volume[:15].plot.barh()
plt.ylabel('assetName')
plt.xlabel('Trading volume')
plt.title('Top 15 asset name by volume')
plt.gca().invert_yaxis()
del assetName_Volume

De la lista, podríamos escoger cualquiera, pero escogeremos las 5 primeras, que además corresponden a distintos sectores y por ello parecen bastante representativas.

Para cada uno de ellos repetiremos la matriz de correlacción del fichero de mercados:

In [ ]:
assetName = 'Bank of America Corp'
sel_market_train_df = market_train_df[market_train_df['assetName']==assetName].sort_values(by='time',ascending=True) 
# Plot correlation
columns_corr = ['returnsOpenNextMktres10', 'volume', 'close', 'open', 'returnsClosePrevRaw1', 'returnsOpenPrevRaw1', 'returnsClosePrevRaw10', 'returnsOpenPrevRaw10',
           'returnsClosePrevMktres1', 'returnsOpenPrevMktres1', 'returnsClosePrevMktres10', 'returnsOpenPrevMktres10']
colormap = plt.cm.RdBu
plt.figure(figsize=(18,15))
sns.heatmap(sel_market_train_df[columns_corr].astype(float).corr(), linewidths=0.1, vmax=1.0, vmin=-1., square=True, cmap=colormap, linecolor='white', annot=True)
plt.title('market_train_df correlation matrix')

In [ ]:
assetName = 'Ford Motor Co'
sel_market_train_df = market_train_df[market_train_df['assetName']==assetName].sort_values(by='time',ascending=True) 
# Plot correlation
columns_corr = ['returnsOpenNextMktres10', 'volume', 'close', 'open', 'returnsClosePrevRaw1', 'returnsOpenPrevRaw1', 'returnsClosePrevRaw10', 'returnsOpenPrevRaw10',
           'returnsClosePrevMktres1', 'returnsOpenPrevMktres1', 'returnsClosePrevMktres10', 'returnsOpenPrevMktres10']
colormap = plt.cm.RdBu
plt.figure(figsize=(18,15))
sns.heatmap(sel_market_train_df[columns_corr].astype(float).corr(), linewidths=0.1, vmax=1.0, vmin=-1., square=True, cmap=colormap, linecolor='white', annot=True)
plt.title('market_train_df correlation matrix')

In [ ]:
assetName = 'General Electric Co'
sel_market_train_df = market_train_df[market_train_df['assetName']==assetName].sort_values(by='time',ascending=True) 
# Plot correlation
columns_corr = ['returnsOpenNextMktres10', 'volume', 'close', 'open', 'returnsClosePrevRaw1', 'returnsOpenPrevRaw1', 'returnsClosePrevRaw10', 'returnsOpenPrevRaw10',
           'returnsClosePrevMktres1', 'returnsOpenPrevMktres1', 'returnsClosePrevMktres10', 'returnsOpenPrevMktres10']
colormap = plt.cm.RdBu
plt.figure(figsize=(18,15))
sns.heatmap(sel_market_train_df[columns_corr].astype(float).corr(), linewidths=0.1, vmax=1.0, vmin=-1., square=True, cmap=colormap, linecolor='white', annot=True)
plt.title('market_train_df correlation matrix')

In [ ]:
assetName = 'Microsoft Corp'
sel_market_train_df = market_train_df[market_train_df['assetName']==assetName].sort_values(by='time',ascending=True) 
# Plot correlation
columns_corr = ['returnsOpenNextMktres10', 'volume', 'close', 'open', 'returnsClosePrevRaw1', 'returnsOpenPrevRaw1', 'returnsClosePrevRaw10', 'returnsOpenPrevRaw10',
           'returnsClosePrevMktres1', 'returnsOpenPrevMktres1', 'returnsClosePrevMktres10', 'returnsOpenPrevMktres10']
colormap = plt.cm.RdBu
plt.figure(figsize=(18,15))
sns.heatmap(sel_market_train_df[columns_corr].astype(float).corr(), linewidths=0.1, vmax=1.0, vmin=-1., square=True, cmap=colormap, linecolor='white', annot=True)
plt.title('market_train_df correlation matrix')

In [ ]:
assetName = 'Intel Corp'
sel_market_train_df = market_train_df[market_train_df['assetName']==assetName].sort_values(by='time',ascending=True) 
# Plot correlation
columns_corr = ['returnsOpenNextMktres10', 'volume', 'close', 'open', 'returnsClosePrevRaw1', 'returnsOpenPrevRaw1', 'returnsClosePrevRaw10', 'returnsOpenPrevRaw10',
           'returnsClosePrevMktres1', 'returnsOpenPrevMktres1', 'returnsClosePrevMktres10', 'returnsOpenPrevMktres10']
colormap = plt.cm.RdBu
plt.figure(figsize=(18,15))
sns.heatmap(sel_market_train_df[columns_corr].astype(float).corr(), linewidths=0.1, vmax=1.0, vmin=-1., square=True, cmap=colormap, linecolor='white', annot=True)
plt.title('market_train_df correlation matrix')

Comprobamos que con cada compañía la matriz es diferente:
- Siempre se puede comprobar la relación entre las variables "Raw" y "Mktres" correspondiente
- pero, en cuanto a las variables que se presentan como significativas para la variable objetivo:
    a. en un caso la más significativa parece el volúmen de cotización
    b. en otro caso parecen ser las cotizaciones a la apertura y al cierre
    c. en los otros tres casos parecen ser los retornos en los últimos 10 días (tanto "Open" como "Close", "Raw" y "Mktres") 

Lo cuál ya nos da una primera conclusión...
podría tener sentido hacer un modelo por cada compañía y probablemente la capacidad de predicción sea mucho mejor.

No obstante, aún es un poco pronto para pronunciarnos al respecto, pues el propósito fundamental de esta competición es el uso de las noticias para la predicción y, aún estamos únicamente con el fichero de mercados.


Por otro lado, parece que puede resultar interesante abundar en las relaciones detectadas hasta el momento, centrándonos en:
- relaciones entre variables "Raw" y "Mktres"
- y, sobre todo, en relaciones entre la variable objetivo y el resto de variables.

Al comenzar a hacer estas comparaciones, nos damos cuenta de que aparecen ciertos atípicos en los retornos, que no permiten ver bien la relación entre las variables...

Entonces recordamos aquellos valores atípicos que se habían detectado en la variable, open...
y, dado que los retornos se calculan en base a variaciones en los valores open y close entre sesiones, decidimos hacer primero un análisis de la relación entre las variables:
- open
- close

In [ ]:
colors = (0,0,0)
area = np.pi*3
plt.scatter(market_train_df['close'], market_train_df['open'], s=area, c=colors, alpha=0.5)
plt.title('Scatter plot')
plt.xlabel('close')
plt.ylabel('open')
plt.show()

Vemos, efectivamente, cómo hay ciertos valores de open que no parecen lógicos... el valor al inicio de la sesión y al cierre, dentro de que varíen, lo normal es que se asemejen bastante.

Intentemos calcularlo numéricamente:

In [ ]:
market_train_df['price_diff'] = market_train_df['close'] - market_train_df['open']
grouped = market_train_df.groupby('time').agg({'price_diff': ['std', 'min']}).reset_index()
print(f"Average standard deviation of price change within a day in {grouped['price_diff']['std'].mean():.4f}.")

Efectivamente, la desviación estandar media de la diferencia de precio dentro de un día, vemos que es del 0.9916.

Ahora, intentemos detectar esas variaciones en el precio durante un día que no parecen realistas:

In [ ]:
market_train_df['close_to_open'] =  np.abs(market_train_df['close'] / market_train_df['open'])

print(f"In {(market_train_df['close_to_open'] >= 1.2).sum()} lines price increased by 20% or more.")
print(f"In {(market_train_df['close_to_open'] <= 0.8).sum()} lines price decreased by 20% or more.")

print(f"In {(market_train_df['close_to_open'] >= 2).sum()} lines price increased by 100% or more.")
print(f"In {(market_train_df['close_to_open'] <= 0.5).sum()} lines price decreased by 100% or more.")

Vemos que en muy pocos casos la variación es superior al 20%...
y que en menos de 10 casos la variación es de más del 100%.

Decidimos eliminar esos 9 casos antes de seguir adelante.

In [ ]:
market_train_df = market_train_df.loc[market_train_df['close_to_open'] > 0.5]
market_train_df = market_train_df.loc[market_train_df['close_to_open'] < 2]

Comprobamos nuevamente la relación entre open y close:

In [ ]:
colors = (0,0,0)
area = np.pi*3
plt.scatter(market_train_df['close'], market_train_df['open'], s=area, c=colors, alpha=0.5)
plt.title('Scatter plot')
plt.xlabel('close')
plt.ylabel('open')
plt.show()

Y ahora, tras haber eliminado esos atípicos...
procedamos a ver la relación entre los pares:
- "Raw" rendimientos en crudo
- "Mktres" rendimientos ajustados al mercado
... tanto para las variables "Open" como "Close", tanto interdiarias como a 10 días:

In [ ]:
colors = (0,0,0)
area = np.pi*3
plt.scatter(market_train_df['returnsClosePrevRaw1'], market_train_df['returnsClosePrevMktres1'], s=area, c=colors, alpha=0.5)
plt.title('Scatter plot')
plt.xlabel('returnsClosePrevRaw1')
plt.ylabel('returnsClosePrevMktres1')
plt.show()

In [ ]:
colors = (0,0,0)
area = np.pi*3
plt.scatter(market_train_df['returnsClosePrevRaw10'], market_train_df['returnsClosePrevMktres10'], s=area, c=colors, alpha=0.5)
plt.title('Scatter plot')
plt.xlabel('returnsClosePrevRaw10')
plt.ylabel('returnsClosePrevMktres10')
plt.show()

In [ ]:
colors = (0,0,0)
area = np.pi*3
plt.scatter(market_train_df['returnsOpenPrevRaw1'], market_train_df['returnsOpenPrevMktres1'], s=area, c=colors, alpha=0.5)
plt.title('Scatter plot')
plt.xlabel('returnsOpenPrevRaw1')
plt.ylabel('returnsOpenPrevMktres1')
plt.show()

In [ ]:
colors = (0,0,0)
area = np.pi*3
plt.scatter(market_train_df['returnsOpenPrevRaw10'], market_train_df['returnsOpenPrevMktres10'], s=area, c=colors, alpha=0.5)
plt.title('Scatter plot')
plt.xlabel('returnsOpenPrevRaw10')
plt.ylabel('returnsOpenPrevMktres10')
plt.show()

Comprobamos lo que ya apuntaba la matriz de correlaciones, cuando mostraba aquellas diagonales de altas correlaciones entre cada variable de rendimientos en crudo y su correspondiente variable de rendimientos ajustados al mercado.


Ahora, centrémonos en la variable objetivo y en la correlación del resto de variables con ella:

In [ ]:
colors = (0,0,0)
area = np.pi*3
plt.scatter(market_train_df['volume'], market_train_df['returnsOpenNextMktres10'], s=area, c=colors, alpha=0.5)
plt.title('Scatter plot')
plt.xlabel('volume')
plt.ylabel('returnsOpenNextMktres10')
plt.show()

In [ ]:
colors = (0,0,0)
area = np.pi*3
plt.scatter(market_train_df['close'], market_train_df['returnsOpenNextMktres10'], s=area, c=colors, alpha=0.5)
plt.title('Scatter plot')
plt.xlabel('close')
plt.ylabel('returnsOpenNextMktres10')
plt.show()

In [ ]:
colors = (0,0,0)
area = np.pi*3
plt.scatter(market_train_df['open'], market_train_df['returnsOpenNextMktres10'], s=area, c=colors, alpha=0.5)
plt.title('Scatter plot')
plt.xlabel('open')
plt.ylabel('returnsOpenNextMktres10')
plt.show()

In [ ]:
colors = (0,0,0)
area = np.pi*3
plt.scatter(market_train_df['returnsClosePrevRaw1'], market_train_df['returnsOpenNextMktres10'], s=area, c=colors, alpha=0.5)
plt.title('Scatter plot')
plt.xlabel('returnsClosePrevRaw1')
plt.ylabel('returnsOpenNextMktres10')
plt.show()

In [ ]:
colors = (0,0,0)
area = np.pi*3
plt.scatter(market_train_df['returnsOpenPrevRaw1'], market_train_df['returnsOpenNextMktres10'], s=area, c=colors, alpha=0.5)
plt.title('Scatter plot')
plt.xlabel('returnsOpenPrevRaw1')
plt.ylabel('returnsOpenNextMktres10')
plt.show()

In [ ]:
colors = (0,0,0)
area = np.pi*3
plt.scatter(market_train_df['returnsClosePrevRaw10'], market_train_df['returnsOpenNextMktres10'], s=area, c=colors, alpha=0.5)
plt.title('Scatter plot')
plt.xlabel('returnsClosePrevRaw10')
plt.ylabel('returnsOpenNextMktres10')
plt.show()

In [ ]:
colors = (0,0,0)
area = np.pi*3
plt.scatter(market_train_df['returnsOpenPrevRaw10'], market_train_df['returnsOpenNextMktres10'], s=area, c=colors, alpha=0.5)
plt.title('Scatter plot')
plt.xlabel('returnsOpenPrevRaw10')
plt.ylabel('returnsOpenNextMktres10')
plt.show()

In [ ]:
colors = (0,0,0)
area = np.pi*3
plt.scatter(market_train_df['returnsClosePrevMktres1'], market_train_df['returnsOpenNextMktres10'], s=area, c=colors, alpha=0.5)
plt.title('Scatter plot')
plt.xlabel('returnsClosePrevMktres1')
plt.ylabel('returnsOpenNextMktres10')
plt.show()

In [ ]:
colors = (0,0,0)
area = np.pi*3
plt.scatter(market_train_df['returnsOpenPrevMktres1'], market_train_df['returnsOpenNextMktres10'], s=area, c=colors, alpha=0.5)
plt.title('Scatter plot')
plt.xlabel('returnsOpenPrevMktres1')
plt.ylabel('returnsOpenNextMktres10')
plt.show()

In [ ]:
colors = (0,0,0)
area = np.pi*3
plt.scatter(market_train_df['returnsClosePrevMktres10'], market_train_df['returnsOpenNextMktres10'], s=area, c=colors, alpha=0.5)
plt.title('Scatter plot')
plt.xlabel('returnsClosePrevMktres10')
plt.ylabel('returnsOpenNextMktres10')
plt.show()

In [ ]:
colors = (0,0,0)
area = np.pi*3
plt.scatter(market_train_df['returnsOpenPrevMktres10'], market_train_df['returnsOpenNextMktres10'], s=area, c=colors, alpha=0.5)
plt.title('Scatter plot')
plt.xlabel('returnsOpenPrevMktres10')
plt.ylabel('returnsOpenNextMktres10')
plt.show()

Y ahora, la del fichero de noticias:

In [ ]:
# Plot correlation
columns_corr = ['urgency', 'takeSequence', 'companyCount','marketCommentary','sentenceCount',\
           'firstMentionSentence','relevance','sentimentClass','sentimentWordCount','noveltyCount24H',\
           'noveltyCount3D', 'noveltyCount5D', 'noveltyCount7D','volumeCounts24H','volumeCounts3D',\
           'volumeCounts5D','volumeCounts7D']
colormap = plt.cm.RdBu
plt.figure(figsize=(29,27))
sns.heatmap(news_train_df[columns_corr].astype(float).corr(), linewidths=0.1, vmax=1.0, vmin=-1., square=True, cmap=colormap, linecolor='white', annot=True)
plt.title('news_train_df correlation matrix')

Las conclusiones de la matriz de correlación del fichero de noticias son:

● Existen correlaciones obvias entre las variables que se distinguen entre sí sólo por el horizonte temporal hacia el pasado que contemplan (novelty count y volume count), mientras más cerca en el tiempo más correlación entre ellas.

● Otras correlaciones tienen que ver con los volúmenes de la noticia (contadores de número de sentencias, contadores de sentimiento, contadores de número de veces que se menciona un activo), que tienen un relativamente alto nivel de correlación entre sí.

● Es natural que a más larga la noticia, más probable sea que sea relevante para el activo sea y más tokens relevantes se encuentren, y más noticias urgentes haya. No quiere decir esto, sin embargo, que sea relevante para el precio de la cotización, esto se verá cuando analicemos los modelos.

● También se encuentra relación inversa entre los indicadores de número de secuencia con la relevancia. Esto es, a medida que se avanza en la historia la relevancia es menor.

● Hay algunas variables con correlación con la variable de rendimiento objetivo particularmente baja y según el análisis exploratorio previo tampoco parecen conceptualmente muy relevantes, o bien están incluidas en variables nuevas, o bien son redundantes. Esto se tendrá en cuenta a la hora de simplificar el modelo eliminando variables. Estas variables son: bodysize, companycount, sentencecount, wordcount, noveltycountxH/D, headlineTag.

En definitiva, hay algunas correlaciones que recomendarían eliminar algunas variables para modelos lineales: aquellas que se relacionan entre sí por ser la misma variable, pero con distinto horizonte temporal.